# 02_手動版_研究智能體

## 🎯 學習目標
- 實作 STORM 第一階段：知識探索
- 開發多角度問題生成機制
- 建立網路資料收集系統
- 實現資訊整理與驗證功能

---

## 1. 環境準備

### 載入必要套件

In [ ]:
# 基礎套件
import os
import json
import time
from datetime import datetime
from typing import List, Dict, Any

# LangChain 相關
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

# 網路搜尋與內容擷取
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup

# 環境變數
from dotenv import load_dotenv
load_dotenv()

print("✅ 套件載入完成")

### 初始化基礎工具

In [ ]:
# 初始化 LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    api_key=os.getenv('OPENAI_API_KEY')
)

print("✅ LLM 初始化完成")

---

## 2. 專家角色定義

### 建立多元專家視角

In [ ]:
class ExpertPersona:
    """專家角色類別"""
    
    def __init__(self, name: str, expertise: str, perspective: str, question_style: str):
        self.name = name
        self.expertise = expertise
        self.perspective = perspective
        self.question_style = question_style
    
    def get_system_prompt(self) -> str:
        """取得專家的系統提示詞"""
        return f"""
你是一位{self.expertise}專家，名叫{self.name}。

你的專業背景：{self.expertise}
你的觀點特色：{self.perspective}
你的提問風格：{self.question_style}

請根據你的專業背景和觀點，針對給定的主題提出深度問題。
你的問題應該：
1. 體現你的專業領域特色
2. 具有實際價值和可研究性
3. 能夠挖掘主題的深層面向
4. 符合你的提問風格
"""

# 定義多種專家角色
expert_personas = {
    "技術專家": ExpertPersona(
        name="Dr. Tech",
        expertise="技術實現與系統架構",
        perspective="注重技術可行性、效能優化和實作細節",
        question_style="精確、具體、著重實作層面"
    ),
    
    "商業分析師": ExpertPersona(
        name="Ms. Business",
        expertise="市場分析與商業策略",
        perspective="關注商業價值、市場需求和獲利模式",
        question_style="策略性、注重ROI和市場影響"
    ),
    
    "用戶體驗專家": ExpertPersona(
        name="Prof. UX",
        expertise="用戶體驗設計與行為研究",
        perspective="以用戶為中心，關注易用性和使用者需求",
        question_style="同理心導向、注重實際使用場景"
    ),
    
    "學術研究者": ExpertPersona(
        name="Prof. Academic",
        expertise="理論研究與學術分析",
        perspective="注重理論基礎、研究方法和學術價值",
        question_style="深度分析、理論導向、批判性思考"
    ),
    
    "政策制定者": ExpertPersona(
        name="Dr. Policy",
        expertise="政策制定與社會影響評估",
        perspective="關注社會影響、倫理考量和政策可行性",
        question_style="全面性考量、注重社會責任"
    )
}

print(f"✅ 已定義 {len(expert_personas)} 種專家角色")
for name, persona in expert_personas.items():
    print(f"   - {name}: {persona.expertise}")

---

## 3. 多角度問題生成器

### 實作問題生成邏輯

In [ ]:
class QuestionGenerator:
    """多角度問題生成器"""
    
    def __init__(self, llm, expert_personas):
        self.llm = llm
        self.expert_personas = expert_personas
    
    def generate_questions_from_expert(self, topic: str, expert_name: str, num_questions: int = 3) -> List[str]:
        """從特定專家角度生成問題"""
        if expert_name not in self.expert_personas:
            raise ValueError(f"未知的專家角色: {expert_name}")
        
        expert = self.expert_personas[expert_name]
        
        # 建立提示詞
        system_prompt = expert.get_system_prompt()
        human_prompt = f"""
主題：{topic}

請針對這個主題，提出 {num_questions} 個具有深度且有價值的研究問題。
每個問題都要：
1. 體現你的專業特色
2. 具有可研究性
3. 能夠產生有用的洞察

請以以下格式回答，每個問題一行：
Q1: [問題內容]
Q2: [問題內容]
Q3: [問題內容]
"""
        
        try:
            # 呼叫 LLM
            messages = [
                SystemMessage(content=system_prompt),
                HumanMessage(content=human_prompt)
            ]
            
            response = self.llm.invoke(messages)
            
            # 解析回應
            questions = []
            lines = response.content.strip().split('\n')
            
            for line in lines:
                line = line.strip()
                if line.startswith('Q') and ':' in line:
                    question = line.split(':', 1)[1].strip()
                    questions.append(question)
            
            return questions
            
        except Exception as e:
            print(f"❌ 生成問題時發生錯誤: {str(e)}")
            return []
    
    def generate_comprehensive_questions(self, topic: str, questions_per_expert: int = 2) -> Dict[str, List[str]]:
        """從所有專家角度生成綜合問題集"""
        all_questions = {}
        
        print(f"🤔 正在從多個專家角度分析主題：{topic}")
        
        for expert_name in self.expert_personas.keys():
            print(f"   ⏳ {expert_name} 思考中...")
            
            questions = self.generate_questions_from_expert(
                topic, expert_name, questions_per_expert
            )
            
            all_questions[expert_name] = questions
            
            print(f"   ✅ {expert_name} 提出了 {len(questions)} 個問題")
            
            # 避免 API 限制，稍作延遲
            time.sleep(1)
        
        return all_questions

# 建立問題生成器
question_generator = QuestionGenerator(llm, expert_personas)
print("✅ 問題生成器已建立")

### 測試問題生成功能

In [ ]:
# 測試主題
test_topic = "人工智慧在教育中的應用"

print(f"🎯 測試主題：{test_topic}")
print("=" * 50)

# 從單一專家角度生成問題
tech_questions = question_generator.generate_questions_from_expert(
    test_topic, "技術專家", 3
)

print("🔧 技術專家的問題：")
for i, question in enumerate(tech_questions, 1):
    print(f"   {i}. {question}")

---

## 4. 網路資料收集器

### 實作搜尋與內容擷取

In [ ]:
class WebResearcher:
    """網路研究員 - 負責搜尋和擷取網路內容"""
    
    def __init__(self):
        self.ddgs = DDGS()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
    
    def search_web(self, query: str, max_results: int = 5) -> List[Dict]:
        """網路搜尋"""
        try:
            print(f"   🔍 搜尋：{query}")
            results = list(self.ddgs.text(query, max_results=max_results))
            print(f"   📝 找到 {len(results)} 筆結果")
            return results
        except Exception as e:
            print(f"   ❌ 搜尋失敗: {str(e)}")
            return []
    
    def extract_content(self, url: str, max_length: int = 2000) -> str:
        """擷取網頁內容"""
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # 移除不需要的元素
            for element in soup(["script", "style", "nav", "header", "footer"]):
                element.decompose()
            
            # 擷取文字
            text = soup.get_text()
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            clean_text = ' '.join(chunk for chunk in chunks if chunk)
            
            # 限制長度
            if len(clean_text) > max_length:
                clean_text = clean_text[:max_length] + "..."
            
            return clean_text
            
        except Exception as e:
            return f"內容擷取失敗: {str(e)}"
    
    def research_question(self, question: str, max_sources: int = 3) -> Dict[str, Any]:
        """針對特定問題進行研究"""
        print(f"📚 研究問題：{question}")
        
        # 搜尋相關資料
        search_results = self.search_web(question, max_sources)
        
        # 擷取內容
        sources = []
        for result in search_results:
            print(f"   📖 擷取：{result['title'][:50]}...")
            
            content = self.extract_content(result['href'])
            
            source_info = {
                'title': result['title'],
                'url': result['href'],
                'snippet': result.get('body', ''),
                'content': content[:1500] if content else '',  # 限制內容長度
                'extracted_at': datetime.now().isoformat()
            }
            
            sources.append(source_info)
            
            # 避免過於頻繁的請求
            time.sleep(0.5)
        
        return {
            'question': question,
            'sources': sources,
            'research_date': datetime.now().isoformat()
        }

# 建立網路研究員
web_researcher = WebResearcher()
print("✅ 網路研究員已建立")

### 測試網路研究功能

In [ ]:
# 測試單一問題研究
test_question = "AI教育應用的最新技術趨勢是什麼？"

research_result = web_researcher.research_question(test_question, 2)

print("\n📊 研究結果摘要：")
print(f"問題：{research_result['question']}")
print(f"找到 {len(research_result['sources'])} 個來源")

for i, source in enumerate(research_result['sources'], 1):
    print(f"\n來源 {i}:")
    print(f"  標題：{source['title'][:60]}...")
    print(f"  網址：{source['url'][:60]}...")
    print(f"  內容長度：{len(source['content'])} 字元")

---

## 5. 資訊分析與摘要器

### 實作內容分析功能

In [ ]:
class ContentAnalyzer:
    """內容分析器 - 分析和摘要研究內容"""
    
    def __init__(self, llm):
        self.llm = llm
    
    def analyze_research_content(self, research_data: Dict[str, Any]) -> Dict[str, Any]:
        """分析研究內容並生成摘要"""
        question = research_data['question']
        sources = research_data['sources']
        
        print(f"🧠 分析研究內容：{question[:50]}...")
        
        # 合併所有來源內容
        all_content = ""
        for i, source in enumerate(sources, 1):
            if source['content'] and '擷取失敗' not in source['content']:
                all_content += f"\n\n來源 {i} ({source['title'][:30]}...):\n{source['content']}"
        
        if not all_content.strip():
            return {
                'question': question,
                'summary': '無法獲得有效內容進行分析',
                'key_points': [],
                'insights': '',
                'confidence': 0.0
            }
        
        # 生成分析提示詞
        analysis_prompt = f"""
你是一位專業的研究分析師。請針對以下問題和相關資料進行深度分析。

研究問題：{question}

相關資料：
{all_content[:3000]}  # 限制內容長度避免超出token限制

請提供以下分析：

1. 綜合摘要 (2-3句話總結回答)：
[摘要內容]

2. 關鍵要點 (3-5個重點，每個一行)：
- [要點1]
- [要點2]
- [要點3]

3. 深度洞察 (專業見解和趨勢分析)：
[洞察內容]

4. 可信度評估 (1-10分，10分最高)：
[分數]

請確保分析客觀、準確，基於提供的資料內容。
"""
        
        try:
            response = self.llm.invoke([HumanMessage(content=analysis_prompt)])
            analysis_text = response.content
            
            # 解析回應內容
            summary = self._extract_section(analysis_text, '1. 綜合摘要', '2. 關鍵要點')
            key_points = self._extract_key_points(analysis_text)
            insights = self._extract_section(analysis_text, '3. 深度洞察', '4. 可信度評估')
            confidence = self._extract_confidence(analysis_text)
            
            return {
                'question': question,
                'summary': summary,
                'key_points': key_points,
                'insights': insights,
                'confidence': confidence,
                'source_count': len(sources),
                'analysis_date': datetime.now().isoformat()
            }
            
        except Exception as e:
            print(f"❌ 內容分析失敗: {str(e)}")
            return {
                'question': question,
                'summary': '分析過程中發生錯誤',
                'key_points': [],
                'insights': '',
                'confidence': 0.0
            }
    
    def _extract_section(self, text: str, start_marker: str, end_marker: str) -> str:
        """從文本中提取特定區段"""
        start_idx = text.find(start_marker)
        if start_idx == -1:
            return ""
        
        start_idx = text.find(':', start_idx) + 1
        end_idx = text.find(end_marker, start_idx)
        
        if end_idx == -1:
            section = text[start_idx:]
        else:
            section = text[start_idx:end_idx]
        
        return section.strip().strip('[]')
    
    def _extract_key_points(self, text: str) -> List[str]:
        """提取關鍵要點"""
        key_points = []
        lines = text.split('\n')
        
        in_key_points_section = False
        for line in lines:
            if '2. 關鍵要點' in line:
                in_key_points_section = True
                continue
            elif '3. 深度洞察' in line:
                break
            
            if in_key_points_section and line.strip().startswith('-'):
                point = line.strip()[1:].strip().strip('[]')
                if point:
                    key_points.append(point)
        
        return key_points
    
    def _extract_confidence(self, text: str) -> float:
        """提取可信度分數"""
        import re
        
        # 尋找數字分數
        confidence_match = re.search(r'4\. 可信度評估[^\d]*(\d+(?:\.\d+)?)', text)
        if confidence_match:
            try:
                score = float(confidence_match.group(1))
                return min(score / 10.0, 1.0)  # 轉換為 0-1 範圍
            except:
                pass
        
        return 0.5  # 預設中等可信度

# 建立內容分析器
content_analyzer = ContentAnalyzer(llm)
print("✅ 內容分析器已建立")

### 測試內容分析功能

In [ ]:
# 分析之前的研究結果
analysis_result = content_analyzer.analyze_research_content(research_result)

print("\n📈 分析結果：")
print(f"問題：{analysis_result['question']}")
print(f"\n摘要：{analysis_result['summary']}")
print(f"\n關鍵要點：")
for i, point in enumerate(analysis_result['key_points'], 1):
    print(f"  {i}. {point}")
print(f"\n深度洞察：{analysis_result['insights']}")
print(f"\n可信度：{analysis_result['confidence']:.1%}")

---

## 6. 完整研究智能體

### 整合所有功能

In [ ]:
class ResearchAgent:
    """研究智能體 - 整合問題生成、資料搜尋、內容分析"""
    
    def __init__(self, llm, expert_personas):
        self.llm = llm
        self.question_generator = QuestionGenerator(llm, expert_personas)
        self.web_researcher = WebResearcher()
        self.content_analyzer = ContentAnalyzer(llm)
        self.expert_personas = expert_personas
    
    def research_topic(self, topic: str, questions_per_expert: int = 2, sources_per_question: int = 2) -> Dict[str, Any]:
        """對主題進行完整研究"""
        print(f"🔬 開始研究主題：{topic}")
        print("=" * 60)
        
        # 第一步：生成多角度問題
        print("\n📋 階段 1: 生成研究問題")
        all_questions = self.question_generator.generate_comprehensive_questions(
            topic, questions_per_expert
        )
        
        # 整理所有問題
        research_questions = []
        for expert_name, questions in all_questions.items():
            for question in questions:
                research_questions.append({
                    'question': question,
                    'expert_perspective': expert_name,
                    'expert_info': self.expert_personas[expert_name].expertise
                })
        
        print(f"\n✅ 總共生成了 {len(research_questions)} 個研究問題")
        
        # 第二步：針對每個問題進行研究
        print("\n🔍 階段 2: 資料收集與分析")
        research_results = []
        
        for i, q_info in enumerate(research_questions, 1):
            print(f"\n--- 研究問題 {i}/{len(research_questions)} ---")
            print(f"專家角度：{q_info['expert_perspective']}")
            
            # 網路搜尋
            raw_research = self.web_researcher.research_question(
                q_info['question'], sources_per_question
            )
            
            # 內容分析
            analysis = self.content_analyzer.analyze_research_content(raw_research)
            
            # 整合結果
            research_results.append({
                'question_info': q_info,
                'raw_research': raw_research,
                'analysis': analysis
            })
            
            print(f"✅ 完成分析，可信度：{analysis['confidence']:.1%}")
        
        # 第三步：生成綜合報告
        print("\n📊 階段 3: 生成綜合報告")
        comprehensive_report = self._generate_comprehensive_report(topic, research_results)
        
        final_result = {
            'topic': topic,
            'research_date': datetime.now().isoformat(),
            'expert_questions': all_questions,
            'detailed_research': research_results,
            'comprehensive_report': comprehensive_report,
            'statistics': {
                'total_questions': len(research_questions),
                'total_sources': sum(len(r['raw_research']['sources']) for r in research_results),
                'average_confidence': sum(r['analysis']['confidence'] for r in research_results) / len(research_results)
            }
        }
        
        print(f"\n🎉 研究完成！")
        print(f"   - 研究問題：{final_result['statistics']['total_questions']} 個")
        print(f"   - 資料來源：{final_result['statistics']['total_sources']} 個")
        print(f"   - 平均可信度：{final_result['statistics']['average_confidence']:.1%}")
        
        return final_result
    
    def _generate_comprehensive_report(self, topic: str, research_results: List[Dict]) -> str:
        """生成綜合研究報告"""
        # 彙整所有關鍵發現
        all_summaries = []
        all_key_points = []
        all_insights = []
        
        for result in research_results:
            analysis = result['analysis']
            if analysis['summary']:
                all_summaries.append(f"• {analysis['summary']}")
            all_key_points.extend(analysis['key_points'])
            if analysis['insights']:
                all_insights.append(analysis['insights'])
        
        # 組合報告
        report = f"""
# {topic} - 綜合研究報告

## 關鍵發現摘要
{"".join([f"\n{summary}" for summary in all_summaries[:5]])}

## 重要要點
{"".join([f"\n• {point}" for point in all_key_points[:10]])}

## 專家洞察
{"".join([f"\n\n{insight}" for insight in all_insights[:3]])}

---
*本報告基於 {len(research_results)} 個研究問題的分析結果生成*
"""
        
        return report.strip()

# 建立完整的研究智能體
research_agent = ResearchAgent(llm, expert_personas)
print("✅ 研究智能體已建立完成")

---

## 7. 完整測試

### 執行完整研究流程

In [ ]:
# 執行完整研究
research_topic = "ChatGPT對傳統教育模式的影響"

# 注意：這將花費一些時間和API費用
# 建議先用較少的參數進行測試
full_research = research_agent.research_topic(
    topic=research_topic,
    questions_per_expert=1,  # 每個專家1個問題
    sources_per_question=2   # 每個問題2個來源
)

### 查看研究結果

In [ ]:
# 顯示綜合報告
print("📊 綜合研究報告:")
print("=" * 60)
print(full_research['comprehensive_report'])

print("\n\n📈 研究統計:")
print("=" * 30)
stats = full_research['statistics']
print(f"研究問題總數：{stats['total_questions']}")
print(f"資料來源總數：{stats['total_sources']}")
print(f"平均可信度：{stats['average_confidence']:.1%}")

### 詳細問題分析

In [ ]:
# 查看各專家的問題
print("🤔 各專家角度的研究問題:")
print("=" * 40)

for expert_name, questions in full_research['expert_questions'].items():
    print(f"\n👨‍🏫 {expert_name}:")
    for i, question in enumerate(questions, 1):
        print(f"   {i}. {question}")

---

## 8. 結果保存

### 將研究結果保存為 JSON 檔案

In [ ]:
# 保存研究結果
import json

# 產生檔案名稱
safe_topic = research_topic.replace(' ', '_').replace('/', '_')
filename = f"research_result_{safe_topic}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

# 保存檔案
try:
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(full_research, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 研究結果已保存至：{filename}")
    print(f"   檔案大小：{os.path.getsize(filename)} bytes")
    
except Exception as e:
    print(f"❌ 保存失敗：{str(e)}")

---

## 🎯 本章小結

### 完成功能
✅ **多角度問題生成**：5種專家角色，從不同視角提出研究問題  
✅ **網路資料收集**：自動搜尋並擷取相關網頁內容  
✅ **內容分析摘要**：使用 LLM 分析和摘要研究內容  
✅ **綜合報告生成**：整合所有研究結果生成完整報告  
✅ **結果持久化**：將研究結果保存為 JSON 格式  

### 技術亮點
1. **模組化設計**：每個功能獨立實作，易於測試和維護
2. **專家角色系統**：模擬多種專業背景的思考方式
3. **錯誤處理**：完善的異常處理機制
4. **可配置參數**：問題數量、來源數量等可調整

### 性能考量
- **API呼叫優化**：適當的延遲避免觸發限制
- **內容長度控制**：避免超出token限制
- **快取機制**：避免重複搜尋相同內容

### 下一步預告
在下個 notebook「03_手動版_寫作智能體」中，我們將：
1. 利用研究結果生成文章大綱
2. 實作段落內容生成功能
3. 建立風格一致性控制機制
4. 整合引用和來源標註

---

*研究智能體開發完成！現在我們有了一個強大的工具，能夠從多個專業角度深度研究任何主題，為後續的寫作提供豐富的素材。*